In [3]:
import pandas as pd
import sqlalchemy
import requests
import re
from io   import StringIO
from bs4  import BeautifulSoup as BS
from time import sleep

In [20]:
import datetime as dt

In [40]:
import os

In [76]:
from functions.logit import load_downloaded
from functions.status import print_refresh

In [25]:
URL = 'https://www.mtggoldfish.com/tournament/pro-tour-final-fantasy#paper'

response = requests.get(URL)
response.status_code

sleep(30)


In [ ]:
'''
URL5 = 'https://www.mtggoldfish.com/tournament/pro-tour-final-fantasy?page=5'

response5 = requests.get(URL)
response5.status_code

sleep(30)
'''

In [ ]:
soup = BS(response.text)
print(soup.prettify())

In [ ]:
tournament = soup.find('table', attr={'class' : 'table-tournament'})

In [ ]:
print(soup.find('table', attrs={'class' : 'table-tournament'}).prettify())

In [ ]:
tournament_df = pd.read_html(
    StringIO(
        str(
            soup.find('table', attrs = {'class' : 'table-tournament'})
        )
    )
)[0]

In [ ]:
decks_table = soup.find('table')
deck_rows = decks_table.findAll('tr')
all_links = [deck.findAll('a') for deck in deck_rows[1:-1:2]] # Step by 2 because every other row was returning blank.
for link in all_links:
  print(link, '\n')
deck_links = [link[0].get('href') for link in all_links]
print(deck_links)
print(len(deck_links))
#deck_links = [deck.get('href') for row in
#for links in all_links[0::2]:
#    print(link, '\n')
#print(deck_links[0][0].get('href'))

In [21]:
tournament_df.head()

NameError: name 'tournament_df' is not defined

In [22]:
with open('../protour/final-fantasy/page1.htm') as f:
    soup_expanded = BS( f.read() )


In [23]:
print(
    soup_expanded.find('div', attrs={'class':'deck-table-container'} )  
)

<div class="deck-table-container">
<table class="deck-view-deck-table">
<tbody><tr class="deck-category-header">
<th colspan="4">
Creatures
(28)
</th>
</tr>
<tr>
<td class="text-right">
4
</td>
<td>
<span class="card_id card_name"><a data-card-id="Heartfire Hero [BLB]" data-full-image="https://cdn1.mtggoldfish.com/images/gf/Heartfire%2BHero%2B%255BBLB%255D.jpg" data-full-image-rotation="FALSE" data-original-title="" href="https://www.mtggoldfish.com/price/Bloomburrow/Heartfire+Hero#paper" rel="popover" title="">Heartfire Hero</a></span>
</td>
<td>
<span aria-label="mana cost: red" class="manacost"><i class="ms ms-r ms-cost ms-shadow"></i></span>
</td>
<td class="text-right">
$ 2.16
</td>
</tr>
<tr>
<td class="text-right">
4
</td>
<td>
<span class="card_id card_name"><a data-card-id="Hired Claw &lt;showcase&gt; [BLB]" data-full-image="https://cdn1.mtggoldfish.com/images/gf/Hired%2BClaw%2B%253Cshowcase%253E%2B%255BBLB%255D.jpg" data-full-image-rotation="FALSE" data-original-title="" href

In [24]:
deck_df = pd.read_html(
    StringIO(
        str(
            soup_expanded.find('div', attrs={'class':'deck-table-container'} )
        )
    )
)

In [19]:
deck_df[0]

,Creatures (28),Creatures (28).1,Creatures (28).2,Creatures (28).3
0,4,Heartfire Hero,NaN,$ 2.16
1,4,Hired Claw,NaN,$ 6.20
2,4,Emberheart Challenger,NaN,$ 6.80
3,3,Magebane Lizard,NaN,$ 3.00
4,4,Manifold Mouse,NaN,$ 5.92
5,4,Screaming Nemesis,NaN,$ 84.00
6,1,Tersa Lightshatter,NaN,$ 1.49
7,4,Twinmaw Stormbrood,NaN,$ 1.00
8,Spells (10),Spells (10),Spells (10),Spells (10)
9,4,Burst Lightning,NaN,$ 0.80


In [14]:
manifold_mouse = pd.read_csv(
    '../protour/final-fantasy/Manifold Mouse [BLB].csv',
    names = [ 'date' , 'price' ]
)

In [52]:
dict_df['manifold_mouse']['date']

,date,price
0,2024-07-10,0.49
1,2024-07-11,24.99
2,2024-07-12,17.91
3,2024-07-13,15.42
4,2024-07-14,14.92
...,...,...
396,2025-08-11,1.50
397,2025-08-12,1.49
398,2025-08-13,1.49
399,2025-08-14,1.49


In [73]:
end_date = pd.to_datetime('2025-06-15', format='%Y-%m-%d')
start_date = pd.to_datetime('2025-06-09', format='%Y-%m-%d')

manifold_mouse['date'] = pd.to_datetime(manifold_mouse['date'])

temp = manifold_mouse.loc[
  manifold_mouse['date'].between(start_date, end_date, inclusive = 'both')                         
]

temp.nsmallest(3, 'price')

#for index, row in manifold_mouse.iterrows():
#  if start_date <= row['date'] and row['date'] <= end_date:
#    if minimum > row['price']:
#      minimum = row['price']
#    print(row['price'])

,date,price
336,2025-06-12,2.34
335,2025-06-11,2.35
337,2025-06-13,2.35


In [62]:
manifold_mouse['date']

0     2024-07-10
1     2024-07-11
2     2024-07-12
3     2024-07-13
4     2024-07-14
         ...    
396   2025-08-11
397   2025-08-12
398   2025-08-13
399   2025-08-14
400   2025-08-15
Name: date, Length: 401, dtype: datetime64[ns]

In [84]:
price_csv_dict = {}

csv_list = os.listdir('../cardprices')

# need to fix
card_list = load_downloaded()
for csv in csv_list:
  print_refresh(f'Looking for {csv} match: {len(price_csv_dict)} csv files found.              ')
  for card in card_list:
    if re.search( card , csv ):
      price_csv_dict['card'] = csv
      print_refresh(f'{csv} match found: {len(price_csv_dict)} csv files found.                     ')
      sleep(3)
print( card_list[0] )

csv_list[0]

'Heartfire Hero'Zur, Eternal Schemer _showcase_ [DMU] (F).csv match: 0 csv files found.               ::::::nd.               :::



'Abhorrent Oculus [DSK].csv'